In [9]:
!pip install tensorboardX
!pip install ml_collections
!pip install monai
!pip install torchio
!pip install tensorflow===2.15.0
!git clone --branch training https://github.com/lesnikowka/TRUNet.git

fatal: destination path 'TRUNet' already exists and is not an empty directory.


In [10]:
%autosave 60

import os
import sys
import random
import numpy as np
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import MaxPool3D
from keras.layers import UpSampling3D
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPool2D
from keras.layers import UpSampling2D
from keras.layers import Concatenate
from keras.layers import Flatten
from google.colab import drive
import cv2
import torchio as tio
import torch
import gc
import keras.backend as K
from datetime import datetime
from google.colab import files
import zipfile
import random
from skimage.transform import resize

Autosaving every 60 seconds


In [11]:
import os
print(os.getcwd())
import sys
sys.path.append('/content/TRUNet')

/content


In [12]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
archive = "2048cropped.zip"
!cp -r drive/MyDrive/arrays/$archive /content/sample_data/$archive
zipfile.ZipFile("/content/sample_data/" + archive, 'r').extractall("/content/sample_data")

In [14]:
NUM_CLASSES = 8
IMAGE_SIZE = 96

In [19]:
import argparse
import ml_collections
import os
from glob import glob
import torch
import numpy as np
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric
from TRUNet.TRUNet_network.model.ViT import VisionTransformer3d as TransUNet3d
from datetime import datetime
from TRUNet_network.trunet_train import trainer
from torchvision import transforms
from TRUNet_network.augmentations import RandomGenerator3d_zoom, Reshape3d_zoom

now = datetime.now()

def TransUNet_configs(img_size):
    configs_trunet = ml_collections.ConfigDict()

    configs_trunet.resnet = ml_collections.ConfigDict()
    configs_trunet.resnet.num_layers = (3, 4, 9)
    configs_trunet.resnet.width_factor = 1
    configs_trunet.transformer_mlp_dim = 3072
    configs_trunet.transformer_num_heads = 12
    configs_trunet.transformer_num_layers = 12
    configs_trunet.transformer_attention_dropout_rate = 0.0
    configs_trunet.transformer_dropout_rate = 0.1
    configs_trunet.classifier = 'seg'
    configs_trunet.decoder_channels = (256, 128, 64, 16)
    configs_trunet.n_classes = 7
    configs_trunet.n_skip = 3
    configs_trunet.skip_channels = [512, 256, 64, 16]
    configs_trunet.patches = ml_collections.ConfigDict()
    configs_trunet.patches.grid = None

    configs_trunet.hidden_size = 768
    configs_trunet.patches.size = 16

    configs_trunet.patch_size = configs_trunet.patches.size  # (results in 14 by 14 grid of patches for input size 224)

    configs_trunet.patches.grid = (
        int(img_size / configs_trunet.patches.size), int(img_size / configs_trunet.patches.size),
        int(img_size / configs_trunet.patches.size))
    configs_trunet.hybrid = True

    return configs_trunet

class fetch_dataset:
    def __init__(self, base_dir, max_num = 5, transform=None):
        print("init start")
        self.transform = transform
        self.data_dir = base_dir
        sample_list = sorted(glob(os.path.join(base_dir, '*.npy')))
        sample_list = [os.path.join(base_dir, str(i) + ".npy") for i in range(max_num)]
        self.sample_list = sample_list
        self.name = [str(i) for i in range(max_num)]
        self.pt = [i for i in range(max_num)]
        print("end")

    def __len__(self):
        print("get length dataset")
        return len(self.sample_list)

    def __getitem__(self, idx):
        print("geiitem idx = ", idx)
        data_path = self.sample_list[idx]
        print("datapath = ", data_path)
        data = np.load(data_path)
        print("size: ", data.shape)
        image, label = data['arr_0'], data['arr_1']
        print("image size: ", image.shape, " label: ", label.shape)


        sample = {'image': image, 'label': label, 'case_name': str(idx)}
        if self.transform:
            sample = self.transform(sample)

        return sample


if __name__ == "__main__":

    args = ml_collections.ConfigDict()
    args.max_epochs = 2
    args.save_path = "save_path"
    args.root_path = "sample_data"
    args.num_classes = NUM_CLASSES
    args.batch_size = 1
    args.seed = 42
    args.base_lr = 0.01

    # Transforms & Augmentations
    train_transforms = transforms.Compose(
        [RandomGenerator3d_zoom(output_size=(IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE))])
    val_transforms = transforms.Compose([Reshape3d_zoom(output_size=[IMAGE_SIZE, IMAGE_SIZE, IMAGE_SIZE])])

    # Define model
    config_net = TransUNet_configs(IMAGE_SIZE)
    config_net.n_classes = args.num_classes
    model = TransUNet3d(config_net, img_size=IMAGE_SIZE, zero_head=False, vis=False)

    config = {'ds_val': fetch_dataset(base_dir=os.path.join(args.root_path, 'images'), max_num=5, transform=val_transforms),
              'ds_train': fetch_dataset(base_dir=os.path.join(args.root_path, 'masks'), max_num=5, transform=train_transforms),
              'loss_function': DiceCELoss(include_background=False, to_onehot_y=True, softmax=True),
              'metric': DiceMetric(include_background=False, reduction="mean"),
              'optimizer': torch.optim.Adam(model.parameters(), args.base_lr),
              'save_interval': 50}

    trainer(args, config, model, args.save_path)

init start
end
init start
end
get length dataset
get length dataset
get length dataset
get length dataset
get length dataset
get length dataset
geiitem idx = geiitem idx = geiitem idx = geiitem idx = geiitem idx =      0

14
23datapath = 
datapath = datapath =  
datapath = datapath =   sample_data/masks/0.npy get length dataset
get length dataset
get length dataset
 sample_data/masks/1.npysample_data/masks/3.npy
sample_data/masks/2.npy
sample_data/masks/4.npy


size:  (96, 224, 224, 1)
size: size: size:  (96, 224, 224, 1) 
size:  (96, 224, 224, 1) (96, 224, 224, 1)
(96, 224, 224, 1)



IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-19-52b97c054ee8>", line 70, in __getitem__
    image, label = data['arr_0'], data['arr_1']
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
